In [1]:
#!pip install torch_sparse

In [2]:
import scipy.sparse
import sklearn.linear_model

"""Define graph forming functions"""
# Import packages

import joblib
import itertools
import numpy as np
import pandas as pd
from datetime import date
from tqdm import tqdm
from functools import partial
#from google.colab import files

from sklearn import tree
import sklearn.svm as svm
from sklearn import ensemble
import sklearn.metrics as skm
import sklearn.preprocessing as pp
import sklearn.linear_model as lms
import sklearn.neural_network as skl_nn
import sklearn.neighbors as neighbors
from sklearn.naive_bayes import GaussianNB
import sklearn.model_selection as model_sel

import torch

#import torch_sparse
import torch.nn as nn
# from torch_geometric.data import Data
# from torch_geometric.nn import GCNConv

In [3]:
# Input data
random_state = 2
np.random.seed(random_state)
data_path = 'C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\models\datasets/'#"/content/datasets/"

max_test_set = 100000

In [4]:
def filter_for_start_yr(df, start_col, end_col) -> pd.DataFrame:
    """Convert dataframe of plants with entry for each year into dataframe with row for each year"""
    # Get rid of emissions for years before start year
    df['Age'] = df['Year'].astype(int) - df[start_col].astype(int)
    df = df[df['Age'] >= 0]

    # Get rid of emissions for years after end years
    df['ToGo'] = df[end_col].astype(int) - df['Year'].astype(int)
    df = df[df['ToGo'] >= 0]

    df[['START_YR', 'END_YR']] = df[['START_YR', 'END_YR']].astype(int)

    return df.drop(columns=['ToGo'])

def pivot_data_dfs(df:pd.DataFrame, time_col) -> [pd.DataFrame, pd.DataFrame]:
    """Pivot data dataframes to get each entry and all year/month values in rows"""
    feature_cols = [i for i in df.columns if i not in [time_col, 'Emissions']]
    df_pivoted = df.pivot(index=feature_cols,
                          columns=time_col,
                          values = 'Emissions').reset_index()
    return df_pivoted, feature_cols


def melt_data_dfs(df:pd.DataFrame, feature_cols, time_col) -> pd.DataFrame:
    """Melt data dataframes to get row per date entry for each facility"""
    melted = df.melt(id_vars=feature_cols, var_name=time_col, value_name='Emissions').dropna(subset=['Emissions'])
    melted[time_col] = melted[time_col].astype(int)
    
    if 'START_YR' in df.columns and 'END_YR' in df.columns:
        melted = filter_for_start_yr(melted, 'START_YR', 'END_YR')

    return melted

## Convert train and test sets into ML ready sets
def series_to_bins(series:pd.Series, bins:list=None, labels:list=None, positive:bool=True):
    # Convert a continuous pandas dataframe column into discrete bins
    if bins is None:
        bin_series = series[series!=0] if positive else series
        bins = [min(bin_series.min(),0)-0.01, bin_series.quantile(0.25), bin_series.quantile(0.5), bin_series.quantile(0.75), bin_series.max()+0.01]
    if labels is None: labels = list(range(len(bins)-1))

    transformer = pp.FunctionTransformer(
        pd.cut, kw_args={'bins': bins, 'labels': labels, 'retbins': False}
    )
    return bins, transformer.fit_transform(series)


def preprocess_yearly(train_set, test_set, y_col='Emissions'):
    """Digitise train and test sets"""

    # Create Y
    bins, y_train_clf = series_to_bins(train_set[y_col])
    _, y_test_clf = series_to_bins(test_set[y_col], bins=[test_set[y_col].min()-0.01]+bins[1:-1]+[test_set[y_col].max()+0.01])

    y_train_reg, y_test_reg = train_set[y_col], test_set[y_col]
    train_set, test_set = train_set.drop(columns=[y_col]), test_set.drop(columns=[y_col])

    # Create X
    # Deal with string columns
    x_enc = pp.OrdinalEncoder()
    string_cols = list(train_set.select_dtypes(include='object').columns)
    train_set[string_cols] = train_set[string_cols].astype(str)
    test_set[string_cols] = test_set[string_cols].astype(str)
    x_strings = pd.concat((train_set[string_cols], test_set[string_cols]))
    x_enc.fit(x_strings)

    # Make float columns into int columns
    float_cols = list(train_set.select_dtypes(include='float').columns)
    train_set[float_cols], test_set[float_cols] = train_set[float_cols].astype(int), test_set[float_cols].astype(int)

    if 'LATITUDE' in list(train_set.columns) and 'LONGITUDE' in list(train_set.columns):
        train_set[['LATITUDE', 'LONGITUDE']] = (train_set[['LATITUDE', 'LONGITUDE']].astype(int)+[90, 180])
        test_set[['LATITUDE', 'LONGITUDE']] = (test_set[['LATITUDE', 'LONGITUDE']].astype(int)+[90, 180])

    int_cols = list(train_set.select_dtypes(include='integer').columns)
    x_ints_min = pd.concat((train_set[int_cols], test_set[int_cols])).min().values
    x_ints_train = train_set[int_cols] - x_ints_min
    x_ints_test = test_set[int_cols] - x_ints_min

    X_train = np.concatenate((x_enc.transform(train_set[string_cols]),
                              x_ints_train.values), axis=1)
    X_test = np.concatenate((x_enc.transform(test_set[string_cols]),
                              x_ints_test.values), axis=1)



    return X_train, X_test, y_train_clf, y_test_clf, y_train_reg, y_test_reg, x_enc

def save_decoded_X(X, x_enc, cols, used, name):
    min_years = [used['START_YR'].astype(int).min(), 1978]
    X_inv = np.concatenate((x_enc.inverse_transform(X[:,:-4]), (X[:,-4:-2]+min_years).astype(int), X[:,-2:]), axis=1)
    pd.DataFrame(X_inv, columns=list(columns[:-2]+['Year']+columns[-2:])).to_csv(name+'.csv')

# Function to split rows into two DataFrames
def split_rows(group, test_fraction):

    num_rows = group.shape[0]
    if num_rows == 1:
        return None, None  # Exclude groups with only one sample

    num_sampled_rows = int(min(max(test_fraction * num_rows, 1), num_rows-1))  # At least one sample for each group

    test_df = group.sample(n=num_sampled_rows)
    train_df = group.drop(test_df.index)

    return train_df, test_df

In [5]:
def custom_interpolate(row):
    if row.count() >= 3:  # Check if there are enough values for polynomial interpolation
        return row.interpolate(method='polynomial', order=order, limit_direction='both')
    else:
        return row.interpolate(method='linear', limit_direction='both')

def metrics(y_true, y_pred, model_type='clf'):
    if model_type == 'clf':
        metric_dict = {'confusion': skm.confusion_matrix(y_true, y_pred),
                       'overall_acc': skm.accuracy_score(y_true, y_pred),
                       'average_acc': skm.balanced_accuracy_score(y_true, y_pred),
                       'kappa': skm.cohen_kappa_score(y_true, y_pred),
                       'IoU': skm.jaccard_score(y_true, y_pred, average='weighted')}
    elif model_type == 'reg':
        metric_dict = {'r2': skm.r2_score(y_true, y_pred),
                       'mae': skm.mean_absolute_error(y_true, y_pred),
                       'mse': skm.mean_squared_error(y_true, y_pred)}

    else: raise 'Incorrect model type'

    return metric_dict

In [77]:
from sklearn.model_selection import GridSearchCV

### Training loop
param_grids_orig = [
    {  # Linear Perceptron
        'penalty': ['l1', 'l2', 'elasticnet'],
        'alpha': [0.0001, 0.001, 0.01],
        'class_weight': [None, 'balanced'],
        'max_iter': [20]
    },
    {  # MLP Classifier
        'hidden_layer_sizes': [(100,), (100, 100)],
        'activation': ['relu'], # 'tanh'
        'alpha': [0.001, 0.01],#0.0001,
        'solver': ['adam'],#, 'lbfgs'],
        'learning_rate': ['constant'],#, 'adaptive']
        'max_iter': [20]
    }
]

## Inputs
# Input data
#data_path = "C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\data\classification_inputs/"
for input_data in ['petrochemicals']:
    print(input_data)
     # Output data
    #data_path = 'C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\models/datasets/'
    model_path = 'C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\models/'+input_data+'/'

    balance = True
    gap_filling_levels = [1,2,3]
    # Define divider for level 3
    if input_data == 'CT_manufacturing':
        divider = 'iso3_country'
        inference_cols = ['iso3_country', 'original_inventory_sector', 'asset_type']
        time_col = 'Timestep'
        timesteps = [str(i) for i in range(0,90)]
        graph_cols, max_edges = [0,1,3,5], 100
    elif input_data == 'petrochemicals':
        divider = 'COUNTRY/TERRITORY'
        inference_cols = ['PRODUCT', 'COUNTRY/TERRITORY']
        time_col = 'Year'
        timesteps = [str(i) for i in range(1978,2051)]
        graph_cols, max_edges = [0,3], 30
    elif input_data == 'unfccc':
        divider='Party'
        inference_cols = ['Party', 'Category']
        time_col = 'Year'
        timesteps = [str(i) for i in range(1990,2021)]
        graph_cols, max_edges = [0], 100

    ## Parameters
    max_test_set = 100000
    random_state = 2
    test_size = 0.3
    regression = False

    # Gap level
    for gap_filling_level in gap_filling_levels:
        print('Gap level '+str(gap_filling_level))
        #feature_cols, train_yearly, test_yearly = create_yearly_data(data, input_data, gap_filling_level, timesteps, test_size)
        # Digitise train & test sets
        #X_train_unscaled, X_test_unscaled, y_train_clf, y_test_clf, y_train_reg, y_test_reg, x_enc = preprocess_yearly(train_yearly, test_yearly)

        # Scale datasets & save as .npy
        # scaler = pp.StandardScaler().fit(X_train_unscaled)
        # X_train, X_test = scaler.transform(X_train_unscaled), scaler.transform(X_test_unscaled)

        X_train = np.load(data_path+'X_train-'+input_data+'-'+str(gap_filling_level)+'.npy')
        X_test = np.load(data_path+'X_test-'+input_data+'-'+str(gap_filling_level)+'.npy')
        y_train_clf = pd.read_csv(data_path+'y_train-'+input_data+'-'+str(gap_filling_level)+'.csv', index_col=0)
        y_test_clf = pd.read_csv(data_path+'y_test-'+input_data+'-'+str(gap_filling_level)+'.csv', index_col=0)

        if balance:
            X_train, X_test, y_train_clf, y_test_clf = balance_classes(X_train, X_test, y_train_clf, y_test_clf)
        # if input_data == 'petrochemicals':
        #     names = ['mlpClassifier']
        #     models = [skl_nn.MLPClassifier()]
        #     param_grids = param_grids_orig[1:]
        # else:
        names = ['linearPerceptron', 'mlpClassifier']
        models = [lms.Perceptron(), skl_nn.MLPClassifier()]
        param_grids = param_grids_orig

        for model_name, model, param_grid in zip(names, models, param_grids):
            print(model_name)
            model_type = 'reg' if regression else 'clf'
            model_file = model_path + model_type + '_' + model_name + '_l' + str(str(gap_filling_level)) + '_' + date.today().strftime("%y%m%d")

            y_train, y_test = y_train_clf, y_test_clf

            # Perform grid search to find the best hyperparameters
            grid_search = GridSearchCV(model, param_grid, scoring='accuracy', cv=5)
            grid_search.fit(X_train, y_train)

            # Get the best model with optimized hyperparameters
            best_model = grid_search.best_estimator_

            best_model.max_iter = 200

            # Train the best model on the training data
            print('Fitting optimal params')
            best_model.fit(X_train, y_train)

            # Save the best model
            #joblib.dump(best_model, model_file + '.joblib')

            # Evaluate the best model on the testing data
            y_pred = best_model.predict(X_test)

            scores = metrics(y_test, y_pred, model_type)

            file_name = model_file + '.npy'
            np.save(file_name, scores)

            #files.download(file_name)

            # Save the accuracies generated by each combination to a file
            with open(model_file + '_accuracies.txt', 'w') as file:
                file.write('Hyperparameters,Accuracy\n')
                for params, accuracy in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score']):
                    file.write(str(params) + ',' + str(accuracy) + '\n')

            #files.download(model_file + '_accuracies.txt')

petrochemicals
Gap level 1
linearPerceptron


C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consid

Fitting optimal params
mlpClassifier


C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimiza

Fitting optimal params


...

C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consid

Fitting optimal params
mlpClassifier


C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimiza

Fitting optimal params


C:\ProgramData\Anaconda3\envs\emissions-tracking-conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [78]:
grid_search.cv_results_

{'mean_fit_time': array([35.69543171, 62.27018056, 34.74829082, 61.94452448]),
 'std_fit_time': array([2.85724055, 4.16841466, 1.14373585, 1.54083078]),
 'mean_score_time': array([0.13722925, 0.25767565, 0.13818951, 0.25914917]),
 'std_score_time': array([0.01070869, 0.02035802, 0.00851035, 0.02699224]),
 'param_activation': masked_array(data=['relu', 'relu', 'relu', 'relu'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_alpha': masked_array(data=[0.001, 0.001, 0.01, 0.01],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_hidden_layer_sizes': masked_array(data=[(100,), (100, 100), (100,), (100, 100)],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array(data=['constant', 'constant', 'constant', 'constant'],
              mask=[False, False, False, False],
        fill_value='?',
  

In [79]:
scores

{'confusion': array([[43574,  6514,  2107,  1133],
        [ 7947, 27586, 12233,  5562],
        [ 4537, 18545, 19775, 10471],
        [ 3029, 10134, 13458, 26707]], dtype=int64),
 'overall_acc': 0.5515020252025202,
 'average_acc': 0.5515020252025202,
 'kappa': 0.40200270027002705,
 'IoU': 0.39180241283551664}

In [85]:
def balance_classes(X_train, X_test, y_train, y_test, col_name = 'Emissions'):
    min_count = y_train.reset_index().groupby(col_name).count().min()
    y_train_df = y_train.reset_index(drop=True).groupby(col_name).sample(min_count.values)
    y_train = y_train_df[col_name].values
    X_train = X_train[y_train_df.index]

    min_count_test = y_test.reset_index().groupby(col_name).count().min()
    y_test_df = y_test.reset_index(drop=True).groupby(col_name).sample(min_count_test.values)
    y_test = y_test_df[col_name].values
    X_test = X_test[y_test_df.index]

    return X_train, X_test, y_train, y_test

In [86]:
## Single run
from sklearn.model_selection import GridSearchCV

### Training loop
param_grids = [
    # {  # Linear Perceptron
    #     'penalty': ['l1', 'l2', 'elasticnet'],
    #     'alpha': [0.0001, 0.001, 0.01],
    #     'class_weight': [None, 'balanced'],
    #     'max_iter': [20]
    # },
    {  # MLP Classifier
        'hidden_layer_sizes': [(100, 100)],
        'activation': ['relu'], # 'tanh'
        'alpha': [0.01],#0.0001,
        'solver': ['adam'],#, 'lbfgs'],
        'learning_rate': ['constant'],#, 'adaptive']
        'max_iter': [20]
    }
]

## Inputs
# Input data
#data_path = "C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\data\classification_inputs/"
for input_data in ['petrochemicals']:
    print(input_data)
     # Output data
    #data_path = 'C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\models/datasets/'
    model_path = 'C:/Users\lukec\PycharmProjects\emissions-tracking-conda\emissions-tracking\models/'+input_data+'/'

    gap_filling_levels = [2]
    # Define divider for level 3
    if input_data == 'CT_manufacturing':
        divider = 'iso3_country'
        inference_cols = ['iso3_country', 'original_inventory_sector', 'asset_type']
        time_col = 'Timestep'
        timesteps = [str(i) for i in range(0,90)]
        graph_cols, max_edges = [0,1,3,5], 100
    elif input_data == 'petrochemicals':
        divider = 'COUNTRY/TERRITORY'
        inference_cols = ['PRODUCT', 'COUNTRY/TERRITORY']
        time_col = 'Year'
        timesteps = [str(i) for i in range(1978,2051)]
        graph_cols, max_edges = [0,3], 30
        gap_filling_levels = [3]
        balance=True
    elif input_data == 'unfccc':
        divider='Party'
        inference_cols = ['Party', 'Category']
        time_col = 'Year'
        timesteps = [str(i) for i in range(1990,2021)]
        graph_cols, max_edges = [0], 100

    ## Parameters
    max_test_set = 100000
    random_state = 2
    test_size = 0.3
    regression = False

    # Gap level
    for gap_filling_level in gap_filling_levels:
        print('Gap level '+str(gap_filling_level))
        #feature_cols, train_yearly, test_yearly = create_yearly_data(data, input_data, gap_filling_level, timesteps, test_size)
        # Digitise train & test sets
        #X_train_unscaled, X_test_unscaled, y_train_clf, y_test_clf, y_train_reg, y_test_reg, x_enc = preprocess_yearly(train_yearly, test_yearly)

        # Scale datasets & save as .npy
        # scaler = pp.StandardScaler().fit(X_train_unscaled)
        # X_train, X_test = scaler.transform(X_train_unscaled), scaler.transform(X_test_unscaled)

        X_train = np.load(data_path+'X_train-'+input_data+'-'+str(gap_filling_level)+'.npy')
        X_test = np.load(data_path+'X_test-'+input_data+'-'+str(gap_filling_level)+'.npy')
        y_train = pd.read_csv(data_path+'y_train-'+input_data+'-'+str(gap_filling_level)+'.csv', index_col=0)
        y_test = pd.read_csv(data_path+'y_test-'+input_data+'-'+str(gap_filling_level)+'.csv', index_col=0)

        if balance:
            X_train, X_test, y_train, y_test = balance_classes(X_train, X_test, y_train, y_test)
        # if input_data == 'petrochemicals':
        #     names = ['mlpClassifier']
        #     models = [skl_nn.MLPClassifier()]
        #     param_grids = param_grids_orig[1:]
        # else:
        #     names = ['linearPerceptron', 'mlpClassifier']
        #     models = [lms.Perceptron(), skl_nn.MLPClassifier()]
        #     param_grids = param_grids_orig

        model = skl_nn.MLPClassifier(max_iter=200, hidden_layer_sizes=(100, 100), activation='relu', alpha=0.01, solver='adam', learning_rate='constant', early_stopping=True, verbose=True)
        model_name = 'mlpClassifier'

        # for model_name, model, param_grid in zip(names, models, param_grids):
        #     print(model_name)
        model_type = 'reg' if regression else 'clf'
        model_file = model_path + model_type + '_' + model_name + '_l' + str(str(gap_filling_level)) + '_' + date.today().strftime("%y%m%d")

        model.fit(X_train, y_train)

        # Save the best model
        #joblib.dump(best_model, model_file + '.joblib')

        # Evaluate the best model on the testing data
        y_pred = model.predict(X_test)

        scores = metrics(y_test, y_pred, model_type)

        file_name = model_file + '.npy'
        np.save(file_name, scores)

        #files.download(file_name)

        # Save the accuracies generated by each combination to a file
        # with open(model_file + '_accuracies.txt', 'w') as file:
        #     file.write('Hyperparameters,Accuracy\n')
        #     for params, accuracy in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score']):
        #         file.write(str(params) + ',' + str(accuracy) + '\n')

        #files.download(model_file + '_accuracies.txt')

petrochemicals
Gap level 3
Iteration 1, loss = 1.00769300
Validation score: 0.608422
Iteration 2, loss = 0.86291125
Validation score: 0.648491
Iteration 3, loss = 0.79929308
Validation score: 0.683605
Iteration 4, loss = 0.75514366
Validation score: 0.698607
Iteration 5, loss = 0.72229795
Validation score: 0.715124
Iteration 6, loss = 0.69671295
Validation score: 0.731117
Iteration 7, loss = 0.67672627
Validation score: 0.736363
Iteration 8, loss = 0.66076216
Validation score: 0.741824
Iteration 9, loss = 0.64579390
Validation score: 0.753153
Iteration 10, loss = 0.63407162
Validation score: 0.757447
Iteration 11, loss = 0.62288305
Validation score: 0.764618
Iteration 12, loss = 0.61355487
Validation score: 0.769340
Iteration 13, loss = 0.60534682
Validation score: 0.770739
Iteration 14, loss = 0.59636047
Validation score: 0.781621
Iteration 15, loss = 0.59018369
Validation score: 0.776549
Iteration 16, loss = 0.58359381
Validation score: 0.778939
Iteration 17, loss = 0.57849099
Valida

In [87]:
scores

{'confusion': array([[44022,  4702,  2637,  1967],
        [ 9781, 20391, 15057,  8099],
        [ 5195, 13745, 19567, 14821],
        [ 2710,  7146, 12450, 31022]], dtype=int64),
 'overall_acc': 0.5391257875787578,
 'average_acc': 0.5391257875787578,
 'kappa': 0.38550105010501046,
 'IoU': 0.3773292134897432}

In [10]:
scores

{'confusion': array([[200430,  13413,   7748,   4517],
        [ 15851,  19522,  12277,   6302],
        [  9091,  13329,  21614,  13581],
        [  5078,   5583,  12143,  30524]], dtype=int64),
 'overall_acc': 0.6958770137313525,
 'average_acc': 0.5489506387386249,
 'kappa': 0.49435134705258865,
 'IoU': 0.5727783096689028}